<a href="https://colab.research.google.com/github/seidikun/nivelamento-icm/blob/main/Exerc%C3%ADcio_3_Implementando_uma_ICM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objetivo: construir um procesamento para reconhecimento de olhos abertos vs olhos fechados em sinal EEG

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Abrir arquivo
2. Pre-processamento (re-referenciar, filtrar, epocar, etc)
3. Extrair caracteristicas
4. Instanciar um modelo de ML

In [ ]:
import pandas as pd
import numpy as np

# Abrindo o dado

In [ ]:
pasta_dados = '/content/drive/Shareddrives/Grupo Ada/🧠 PROJETOS/🦾🧠 ICM com treino zero/Dados/Coleta/Coleta 09 08/'
arquivo     = 'open and close'

df_eyes = pd.read_csv(pasta_dados + arquivo)

In [ ]:
df_eyes.head()

,Time:512Hz,Epoch,FP1,FP2,F3,Fz,T7,C3,Cz,P3,...,Oz,O2,Pz,P4,C4,F4,T8,Event Id,Event Date,Event Duration
0,0.000000,0,27133.742188,19073.292969,23559.962891,10953.663086,-2536.377197,25672.707031,15310.254883,3130.957275,...,-898.730530,-6388.867676,10216.211914,202.099625,12865.381836,14929.444336,10565.430664,NaN,NaN,NaN
1,0.001953,0,27134.962891,19064.503906,23557.130859,10950.245117,-2538.720947,25672.853516,15310.938477,3128.662354,...,-896.093811,-6385.352051,10215.577148,204.541031,12861.622070,14928.418945,10570.801758,NaN,NaN,NaN
2,0.003906,0,27137.355469,19068.068359,23558.058594,10951.954102,-2545.508057,25674.220703,15308.155273,3131.592041,...,-898.828186,-6385.010254,10216.993164,199.902359,12861.573242,14924.268555,10553.711914,NaN,NaN,NaN
3,0.005859,0,27137.404297,19064.259766,23555.714844,10945.850586,-2545.508057,25674.904297,15309.327148,3129.297119,...,-900.439514,-6387.354004,10214.063477,203.417984,12860.108398,14923.340820,10534.327148,NaN,NaN,NaN
4,0.007812,0,27136.183594,19072.853516,23562.160156,10953.125977,-2543.701416,25672.902344,15308.985352,3129.443604,...,-898.242249,-6382.959473,10217.383789,204.394547,12865.381836,14922.754883,10551.319336,NaN,NaN,NaN


# Pré-processamento

## Re-referenciamento: filtro CAR

Ideia: referenciar cada canal em relação ao valor médio de todos os canais

In [ ]:
means = df_eyes.iloc[:,2:18].mean(axis = 1)

for ch in df_eyes.iloc[:,2:18].columns:
  df_eyes[ch] = df_eyes[ch] - means

## Segmentação

Ideia: a partir do dado completo, extrair janelas, ou épocas, do sinal, de forma a obter exemplos de treino e teste para o modelo

In [ ]:
ind_i = 0
ind_f = 5*512
epochs = []
while(ind_f < len(df_eyes)):
  epochs.append(df_eyes.iloc[ind_i:ind_f, 2:18])
  ind_i += 5*512
  ind_f += 5*512

In [ ]:
np.array(epochs).shape

(12, 2560, 16)

In [ ]:
np.array(epochs)[0]

array([[16924.48233509,  8864.03311634, 13350.70303822, ...,
         2656.12198353,  4720.18448353,   356.17081165],
       [16926.30118275,  8855.84219837, 13348.4691515 , ...,
         2652.96036243,  4719.75723743,   362.14004993],
       [16930.14029408,  8860.8531847 , 13350.84341908, ...,
         2654.35806751,  4717.05338001,   346.49673939],
       ...,
       [17018.66568422,  8897.57193422, 13334.68130922, ...,
         2632.67837954,  4673.74283266,   285.16861391],
       [17011.90604067,  8894.37674379, 13330.55838442, ...,
         2631.33865786,  4672.35428286,   285.34256411],
       [17011.23160219,  8891.94449282, 13329.15152407, ...,
         2631.00601625,  4675.19546938,   288.18375063]])

# Extração de características

# Classificação